This tutorial teaches you features in .NET Core and the C# language. You’ll learn how to:

- Generate sequences with LINQ.
- Write methods that can be easily used in LINQ queries.
- Distinguish between eager and lazy evaluation.

You'll learn these techniques by building an application that demonstrates one of the basic skills of any magician: the [faro shuffle](https://en.wikipedia.org/wiki/Faro_shuffle). Briefly, a faro shuffle is a technique where you split a card deck exactly in half, then the shuffle interleaves each one card from each half to rebuild the original deck.

Magicians use this technique because every card is in a known location after each shuffle, and the order is a repeating pattern.

For your purposes, it is a light hearted look at manipulating sequences of data. The application you'll build constructs a card deck and then performs a sequence of shuffles, writing the sequence out each time. You'll also compare the updated order to the original order.

## Create a deck of cards

Commonly, a deck of playing cards has four suits, and each suit has thirteen values. Let's create two `enum` types to represent the suits and the ranks, and two extension methods that enumerate each suit and each rank:

In [1]:
public enum Suit
    {
        Clubs,
        Diamonds,
        Hearts,
        Spades
    }
    
Console.WriteLine(".NET Interactive knows about a Suit");

 public enum Rank
    {
        Two,
        Three,
        Four,
        Five,
        Six,
        Seven,
        Eight,
        Nine,
        Ten,
        Jack,
        Queen,
        King,
        Ace
    }
    
Console.WriteLine(".NET Interactive knows about a Face.");

static IEnumerable<Suit> Suits() => Enum.GetValues(typeof(Suit)) as IEnumerable<Suit>;
static IEnumerable<Rank> Ranks() => Enum.GetValues(typeof(Rank)) as IEnumerable<Rank>;

Console.WriteLine(".NET Interactive can generate suits and ranks.");

Next, define a `Deck` class to represent the deck of cards. This has two constructors. The default constructor generate a deck in the sorted order. The second constructor takes a sequence of cards in the specified order. Later, you'll use the second constructor as you shuffle the deck.

In [2]:

public class Card
{
    public Suit Suit { get; }
    public Rank Rank { get; }

    public Card(Suit s, Rank r)
    {
        Suit = s;
        Rank = r;
    }

    public override string ToString() => $"{Rank} of {Suit}";
}

public class Deck
{
    private List<Card> cards = new List<Card>();
    
    public Deck()
    {
        var sequence = from s in Suits()
            from r in Ranks()
            select new Card(s, r);
        cards.AddRange(sequence);
    }
    public Deck(IEnumerable<Card> cards) => this.cards = new List<Card>(cards);

    public IEnumerable<Card> Cards => cards;
}

Console.WriteLine("The deck of cards is defined");

In [3]:
Formatter<Deck>.Register((deck, writer) =>
{
    PocketView v = div(deck.Cards.Select(c =>
    {
        var face = Enum.GetName(typeof(Rank), c.Rank);
        var suit = Enum.GetName(typeof(Suit), c.Suit);
        return img[src:$"https://dotnetcards.azureedge.net/cards/{suit}{face}.png", width:64]();
    }));
    writer.Write(v);
}, "text/html");

Console.WriteLine("card formatter defined");

In [4]:
static IEnumerable<T> InterleaveSequenceWith<T> (this IEnumerable<T> first, IEnumerable<T> second)
{
    var firstIter = first.GetEnumerator();
    var secondIter = second.GetEnumerator();

    while (firstIter.MoveNext() && secondIter.MoveNext())
    {
        yield return firstIter.Current;
        yield return secondIter.Current;
    }
}

Console.WriteLine("Shuffle method is defined");

In [5]:
static bool SequenceEquals<T>(this IEnumerable<T> first, IEnumerable<T> second)
{
    var firstIter = first.GetEnumerator();
    var secondIter = second.GetEnumerator();

    while (firstIter.MoveNext() && secondIter.MoveNext())
    {
        if (!firstIter.Current.Equals(secondIter.Current))
        {
            return false;
        }
    }

    return true;
}
Console.WriteLine("Sequence comparison method defined");

In [7]:
var deck = new Deck();
var deckDisplay = display(deck);
int numberOfShuffles = 0;
var shuffleDisplay = display($"Number of shuffles: {numberOfShuffles}");

In [8]:
var shuffle = deck;
do
{
    shuffle = new Deck(shuffle.Cards.Take(26).InterleaveSequenceWith(shuffle.Cards.Skip(26)));
    numberOfShuffles++;
    deckDisplay.Update(shuffle);
    shuffleDisplay.Update($"Number of shuffles: {numberOfShuffles}");
    await Task.Delay(3000);
} while(!deck.Cards.SequenceEquals(shuffle.Cards));
